## 04 - Caliper Voxel Method
This workflow tests a neighbour-cell based outlining method loosely based on marching squares.
I also switched out the hough outlining for a [rotating calipers](https://en.wikipedia.org/wiki/Rotating_calipers) inspired method for determining the dominant angles of a building.

*This method is MUCH faster. We're talking 100x over the traditional hough / convex hull methods.* It however, needs more work. The results are too 'voxelated'. Future work will go into mixing the methods in this notebook with those in notebook 03 to simplify rectilinear geometry where possible whiling maintaining angled geometry.

I also relied on RhinoCompute for the output.

In [1]:
from code.clouds import SiteCloud, BldgCloud
import numpy as np
from matplotlib import pyplot as plt

import rhino3dm as r3d
import compute_rhino3d.Util

compute_rhino3d.Util.url = "http://localhost:8081/"


### Clouding formatting with revised site cloud class

In [2]:
test_cld = SiteCloud('data/small_bldgs.las')

ASPRS classification in LAS file

In [3]:
test_cld.available_classifs

['unclassified', 'ground', 'buildings']

Translating the geometry to 0,0,0

In [4]:
test_cld.translate_cloud()

Grabbing the buildings and clustering in plan.

In [5]:
bldgs = test_cld.buildings

In [6]:
split_bldgs = test_cld.cluster_in_plan('buildings')

In [7]:
test_bldg = split_bldgs[4]
test_bldg.shape

(124819, 3)

## Generating building outlines
Using new bldg cloud class that implements neighbourhood look up for edges and calliper method for dominant angles

In [8]:
bldg_cld = BldgCloud(test_bldg, voxel_interval= 0.75)

In [9]:
corner_pts = bldg_cld.order_pts(bldg_cld.level_outlines)

In [10]:
bldg_cld.voxel_cntrs[2]

array([ 1.875,  2.625,  3.375,  4.125,  4.875,  5.625,  6.375,  7.125,
        7.875,  8.625,  9.375, 10.125, 10.875, 11.625, 12.375, 13.125,
       13.875, 14.625, 15.375, 16.125, 16.875, 17.625, 18.375, 19.125,
       19.875, 20.625, 21.375, 22.125, 22.875, 23.625, 24.375, 25.125,
       25.875, 26.625, 27.375, 28.125])

In [11]:
bldg_cld.dom_angle

150.5

## Rhino Polylines and Export
---

In [12]:
model = r3d.File3dm()



rot_angle = np.deg2rad(bldg_cld.dom_angle * -1)

for i in range(len(corner_pts)):
    z_index = bldg_cld.unique_levels[i]
    elevation = bldg_cld.voxel_cntrs[2][z_index]
    loop_crvs = []
    for loop in corner_pts[i]:
        loop_points = []
        for item in loop:
            column = item // bldg_cld.histogram.shape[2]
            row = item % bldg_cld.histogram.shape[2]

            x_coord = bldg_cld.voxel_cntrs[0][column]
            y_coord = bldg_cld.voxel_cntrs[1][row]

            x_rotate = np.cos(rot_angle) * x_coord - np.sin(rot_angle) * y_coord
            y_rotate = np.sin(rot_angle) * x_coord + np.cos(rot_angle) * y_coord


            point = r3d.Point3d(x_rotate, y_rotate, elevation)


            loop_points.append(point)
        
        crv_loop = r3d.Polyline(loop_points)
        model.Objects.AddPolyline(crv_loop)
        loop_crvs.append(crv_loop)

model.Write('test.3dm')



True

The output when extruded looks like this:
![calipers and voxel method](data/images/06-calipers%20and%20voxel.png)